In [1]:
!pip install vaderSentiment

In [12]:
# ******** MATH OPERATIONS ********
import numpy as np
from collections import Counter

# ******** TORCH LIBRARIES ******
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

# ******** DATA MANIPULATION ******
import pandas as pd
from skimage.color import rgb2gray

# ******** DATA VISUALIZATION *****
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
%matplotlib inline
sns.set(style='white', context='notebook')

# ******** MODELING ***************
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from scipy.spatial import Voronoi, voronoi_plot_2d
from scipy.stats import pearsonr
from scipy.optimize import curve_fit
from sklearn.linear_model import Perceptron
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# ******** METRICS ****************
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Reference: coding tutorial 1 notebook on canvas

In [14]:
##### Bring the data in using pandas
filename1 = "scores.csv"
filename2 = "transcripts.csv"
# filename3 = "prosodic_features.csv"
# filename4 = "turker_scores_full_interview_old.csv"

# load the data
df_scores = pd.read_csv(filename1)
df_transcripts = pd.read_csv(filename2,names=["Participant", "Transcript"])
# df_prosodic = pd.read_csv(filename3)
# df_tscores = pd.read_csv(filename4)

# change the number of rows displayed
df_scores.head(10)
df_transcripts.head(10)
# df_prosodic.head(10)
# df_tscores.head(10)

# Reference: data cleaning notebook on canvas

,Participant,Transcript
0,p1,Interviewer: So how are you doing?|Interviewee...
1,p10,Interviewer: So how you doing?|Interviewee: G...
2,p11,Interviewer: So tell me about yourself. |Inte...
3,p12,Interviewer: So how are you doing today?|Inter...
4,p13,Interviewer: How are you doing today?|Intervie...
5,p14,Interviewer: So how are you doing today|Inter...
6,p15,Interviewer: Okay so how are you doing today? ...
7,p16,Interviewer: Great so how are you doing today...
8,p17,Interviewer: How are you doing today?|Intervie...
9,p20,Interviewer: How are you doing today?|Intervie...


In [16]:
# high level data type and data info
df_scores.info()
df_transcripts.info()
# df_prosodic.info()
# df_tscores.info()

# Reference: data cleaning notebook on canvas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Participant  138 non-null    object 
 1   Overall      138 non-null    float64
 2   Excited      138 non-null    float64
dtypes: float64(2), object(1)
memory usage: 3.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Participant  138 non-null    object
 1   Transcript   138 non-null    object
dtypes: object(2)
memory usage: 2.3+ KB


In [34]:
df_transcripts["neg"] = 0.0
df_transcripts["neu"] = 0.0
df_transcripts["pos"] = 0.0
df_transcripts["compound"] = 0.0
analyzer = SentimentIntensityAnalyzer()
# for sentence in df_transcripts.Transcript:
#     vs = analyzer.polarity_scores(sentence)
#     print(vs['neg'])
    #print("{:-<65} {}".format(sentence, str(vs)))

for index, row in df_transcripts.iterrows():
    vs = analyzer.polarity_scores(row.Transcript)
    df_transcripts.loc[index,'neg']= vs['neg']
    df_transcripts.loc[index, 'neu']= vs['neu']
    df_transcripts.loc[index, 'pos']= vs['pos']
    df_transcripts.loc[index,'compound']= vs['compound']

df_transcripts.head()


,Participant,Transcript,neg,neu,pos,compound
0,p1,Interviewer: So how are you doing?|Interviewee...,0.013,0.859,0.128,0.9955
1,p10,Interviewer: So how you doing?|Interviewee: G...,0.026,0.852,0.122,0.9988
2,p11,Interviewer: So tell me about yourself. |Inte...,0.020,0.874,0.105,0.9966
3,p12,Interviewer: So how are you doing today?|Inter...,0.027,0.842,0.131,0.9978
4,p13,Interviewer: How are you doing today?|Intervie...,0.038,0.838,0.124,0.9945
